# Modelo para el reconocimiento de Posturas corporales de arbitraje en esgrima (señas arbitrales)


Basado en https://www.youtube.com/watch?v=doDUihpj6ro&t=7833s

GitHub -> https://github.com/nicknochnack/ActionDetectionforSignLanguage

Crear virtual environement
   ->  conda create --env 

# Instalar Dependencias y Librerias

In [2]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib
#!pip list

# Importar librerias

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Capturar puntos corporales con Mediapipe

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
# funcion para detectar los puntos corporales

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # conversion del color BGR to RGB
    image.flags.writeable = False                  # image is no longer writeable 
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # reversion del color RGB to BGR
    return image, results

In [4]:
# funcion para dibujar los puntos corporales (basica)
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # draw face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # draw pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw left hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw right hand

In [5]:
# funcion para dibujar los puntos corporales (estilizada)

def draw_styled_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # draw face
    
    # draw pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(0,255,255), thickness=3, circle_radius=3)
                             )
    
    # draw left hand
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4), #naranja
                             mp_drawing.DrawingSpec(color=(0,128,255), thickness=3, circle_radius=3) # amarillo
                             )
    
    # draw right hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(255,255,0), thickness=3, circle_radius=3)
                             ) 

In [8]:
cap = cv2.VideoCapture(0)
# set mediapipe model
with mp_holistic.Holistic(static_image_mode=True,model_complexity=1,smooth_landmarks=True,min_detection_confidence=0.3,
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # reed feed
        ret, frame = cap.read()

        # make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        
        # show on screen
        cv2.imshow('OpenCV Feed', image)
        
        # break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [35]:
draw_landmarks(frame, results)

# Extract Keypoint values

In [47]:
results.pose_landmarks

landmark {
  x: 0.47400158643722534
  y: 0.7603145837783813
  z: -1.43235445022583
  visibility: 0.9999111890792847
}
landmark {
  x: 0.49888086318969727
  y: 0.6934146285057068
  z: -1.3761368989944458
  visibility: 0.9998179078102112
}
landmark {
  x: 0.5165291428565979
  y: 0.6902409195899963
  z: -1.3763800859451294
  visibility: 0.9998027682304382
}
landmark {
  x: 0.5322569608688354
  y: 0.6869755983352661
  z: -1.3765376806259155
  visibility: 0.9997903108596802
}
landmark {
  x: 0.4379246234893799
  y: 0.6957972645759583
  z: -1.3715057373046875
  visibility: 0.9997922778129578
}
landmark {
  x: 0.4166436195373535
  y: 0.693983256816864
  z: -1.37140691280365
  visibility: 0.9997756481170654
}
landmark {
  x: 0.3971002995967865
  y: 0.6913236379623413
  z: -1.3711178302764893
  visibility: 0.9997497797012329
}
landmark {
  x: 0.5477348566055298
  y: 0.6939939856529236
  z: -0.9351528286933899
  visibility: 0.9997989535331726
}
landmark {
  x: 0.37517547607421875
  y: 0.70671236

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(1404*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    #return np.concatenate([facer,pose,lh,rh])
    return np.concatenate([pose,lh,rh])

# Setup Folders for Collection

In [8]:
# path for exported data
DATA_PATH = os.path.join('MP_data4')

# actions that we are trying to detect
#actions = np.array(['sin seña'])
actions = np.array(['sin seña','enguardia','listos','adelante','punto derecha','punto izquierda','punto doble'])
# cantidad de secuencias de frames  (videos de la accion) que se extraen
no_sequences = 100
# cantidad de frames de cada video que entran en la deteccion
sequence_length = 30

In [9]:
for action in actions:
    for sequence in range(no_sequences):
    #for sequence in range(85,100):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect keypoint Values for training and testing

In [13]:
cap = cv2.VideoCapture(0)
# set mediapipe model
with mp_holistic.Holistic(static_image_mode=True,model_complexity=1,smooth_landmarks=True,min_detection_confidence=0.3,
                          min_tracking_confidence=0.5) as holistic:
    
    # loop through actions
    for action in actions:
        # loop through sequences aka videos
        for sequence in range(no_sequences):
            #loop through video lenght aka frames/sequence lenght
            for frame_num in range(sequence_length):
    
                # reed feed
                ret, frame = cap.read()

                # make detections
                image, results = mediapipe_detection(frame, holistic)
                #print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Postura: {} numero {}'.format(action, sequence), (15,30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 4, cv2.LINE_AA)
                    # show on screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Postura {} numero {}'.format(action, sequence), (15,30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 4, cv2.LINE_AA)
                    # show on screen
                    cv2.imshow('OpenCV Feed', image)

            
                # Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)


                # break
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'sin seña': 0,
 'enguardia': 1,
 'listos': 2,
 'adelante': 3,
 'punto derecha': 4,
 'punto izquierda': 5,
 'punto doble': 6}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [14]:
np.array(sequences).shape

(700, 30, 258)

In [15]:
np.array(labels).shape

(700,)

In [16]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Partición de los datos

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Build and Train LSTM Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
# Modelo

In [21]:
# model structure
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))


In [22]:
# model compilation
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'] )

In [23]:
model.fit(X_train, y_train, epochs=150, callbacks=[tb_callback])

Epoch 1/150
21/21 [==============================] - 3s 53ms/step - loss: 1.8829 - categorical_accuracy: 0.1974
Epoch 2/150
21/21 [==============================] - 1s 38ms/step - loss: 1.3959 - categorical_accuracy: 0.4774
Epoch 3/150
21/21 [==============================] - 1s 39ms/step - loss: 1.1749 - categorical_accuracy: 0.5792
Epoch 4/150
21/21 [==============================] - 1s 39ms/step - loss: 1.1937 - categorical_accuracy: 0.6464
Epoch 5/150
21/21 [==============================] - 1s 41ms/step - loss: 1.6459 - categorical_accuracy: 0.3939
Epoch 6/150
21/21 [==============================] - 1s 39ms/step - loss: 1.1750 - categorical_accuracy: 0.5443
Epoch 7/150
21/21 [==============================] - 1s 38ms/step - loss: 1.1669 - categorical_accuracy: 0.5588
Epoch 8/150
21/21 [==============================] - 1s 39ms/step - loss: 0.8871 - categorical_accuracy: 0.6312
Epoch 9/150
21/21 [==============================] - 1s 38ms/step - loss: 0.6278 - categorical_accuracy:

KeyboardInterrupt: 

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            82688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 231       
Total params: 237,383
Trainable params: 237,383
Non-trainable params: 0
__________________________________________________

# Make Predictions

In [25]:
res = model.predict(X_test)

In [26]:
actions[np.argmax(res[1])]

'enguardia'

In [27]:
actions[np.argmax(y_test[1])]

'enguardia'

# Save Weights and Load Model

In [28]:
# save model
#model.save('action_arb2.h5')



In [29]:
#delete model
# del model

In [30]:
#load model
# 1. run model structure
# 2. run model compilation

model.load_weights('action_arb2.h5')

# Evaluating using a confusion matrix and accuracy

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:
yhat = model.predict(X_test)

In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[30,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  5]],

       [[30,  0],
        [ 0,  5]],

       [[29,  0],
        [ 0,  6]],

       [[31,  0],
        [ 0,  4]]])

In [36]:
accuracy_score(ytrue, yhat)

1.0

# Test in Real Time

In [37]:
from scipy import stats

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,117,245), (16,117,245), (16,117,245), (16,117,2)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [43]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
#with mp_holistic.Holistic(static_image_mode=True,model_complexity=1,smooth_landmarks=True,min_detection_confidence=0.3,
                          #min_tracking_confidence=0.5) as holistic:
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        #cv2.putText(image, '{} - {:.2f}'.format(sentence[-1:],res[np.argmax(res)]),(3,30), 
         #              cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.SolutionOutputs'>
listos
<class 'mediapipe.python.solution_base.S

punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.pyth

...

punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.pyth

<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_

<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto doble
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class 'mediapipe.python.solution_base.SolutionOutputs'>
punto izquierda
<class

<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'me

<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sin seña
<class 'me